<a href="https://colab.research.google.com/github/shainedl/Papers-Colab/blob/master/Hybrid_Convolutional_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on *A Hybrid Convolutional Variational Autoencoder for Text Generation* (Semeniuta et al, Univweaitat zu Lubeck)

In [0]:
import torch
from torch import nn

In [0]:
class VAE(nn.Module):
  def __init__(self, input_size):
    super(VAE, self).__init__()
    
    self.cnn_layers1 = nn.Sequential(
        # Defining 1st 1D convolution layer
        nn.Conv1D(input_size, 128, kernel_size=3, stride=2),
        nn.BatchNorm1D(128),
        nn.ReLU(),

        # Defining 2nd 1D convolution layer
        nn.Conv1D(128, 256, kernel_size=3, stride=2),
        nn.BatchNorm1D(256),
        nn.ReLU(),  
        
        # Defining 3rd 1D convolution layer
        nn.Conv1D(256, 512, kernel_size=3, stride=2),
        nn.BatchNorm1D(512),
        nn.ReLU(),    

        # Defining 4th 1D convolution layer
        nn.Conv1D(512, 512, kernel_size=3, stride=2),
        nn.BatchNorm1D(512),
        nn.ReLU(),    

        # Defining 5th 1D convolution layer
        nn.Conv1D(512, 512, kernel_size=3, stride=2),
        nn.BatchNorm1D(512),
        nn.ReLU()           
    )

    self.cnn_layers2 = nn.Sequential(
        # Defining 1st 1D deconvolution layer
        nn.ConvTranspose1D(512, 512, kernel_size=3, stride=2),
        nn.BatchNorm1D(512),
        nn.ReLU(),

        # Defining 2nd 1D deconvolution layer
        nn.ConvTranspose1D(512, 512, kernel_size=3, stride=2),
        nn.BatchNorm1D(512),
        nn.ReLU(),    

        # Defining 3rd 1D deconvolution layer
        nn.ConvTranspose1D(512, 512, kernel_size=3, stride=2),
        nn.BatchNorm1D(512),
        nn.ReLU(),    

        # Defining 4th 1D deconvolution layer        
        nn.ConvTranspose1D(512, 256, kernel_size=3, stride=2),
        nn.BatchNorm1D(256),
        nn.ReLU(),    

        # Defining 5th 1D deconvolution layer
        nn.ConvTranspose1D(256, 128, kernel_size=3, stride=2),
        nn.BatchNorm1D(128),
        nn.ReLU()        
    )
  
  def encode(self, x):

  
  def decode(self, z):

  
  def forward(self, x):

  
  def __reparameterize(self, mu, logvar):
    